In [7]:
client_id = 
client_secret = 

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 erhn2ch8n2s9z8c49fafrfc9g0qmfl


In [8]:
oauth_token = 

In [9]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/slakun10"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '512977322', 'login': 'slakun10', 'display_name': 'SLAKUN10', 'type': '', 'broadcaster_type': 'partner', 'description': '', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/7d462614-fd45-405c-a0d7-f628f68549e5-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/dac9bfe1-43ec-4246-a61f-34e27d86de40-channel_offline_image-1920x1080.jpeg', 'view_count': 0, 'created_at': '2020-04-12T14:28:48Z'}]}


In [10]:
data = {'data': [{'id': '512977322', 'login': 'slakun10', 'display_name': 'SLAKUN10', 'type': '', 'broadcaster_type': 'partner', 'description': '', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/7d462614-fd45-405c-a0d7-f628f68549e5-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/dac9bfe1-43ec-4246-a61f-34e27d86de40-channel_offline_image-1920x1080.jpeg', 'view_count': 0, 'created_at': '2020-04-12T14:28:48Z'}]}
data_dict = data['data'][0]
data_dict
id_canal =  list(data_dict.values())[0]
id_canal

'512977322'

In [11]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             20 non-null     int64         
 1   URL del VOD            20 non-null     object        
 2   Nombre del VOD         20 non-null     object        
 3   Duración               20 non-null     object        
 4   Fecha de inicio        20 non-null     datetime64[ns]
 5   Fecha de finalización  20 non-null     datetime64[ns]
 6   Visitas                20 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 6.1 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1890870221,https://www.twitch.tv/videos/1890870221,vemos_a_krü_en_el_mundial_de_rocket_!!_krü_vs_...,1h38m20s,2023-08-05 15:56:39,2023-08-05 15:56:39,91759
1,1890044583,https://www.twitch.tv/videos/1890044583,vemos_a_krü_en_el_mundial_de_rocket_!!_krü_vs_...,1h49m10s,2023-08-04 17:29:42,2023-08-04 17:29:42,169550
2,1889178302,https://www.twitch.tv/videos/1889178302,anuncio_de_jugadores_|_kings_y_queens_league,2h45m6s,2023-08-03 17:34:27,2023-08-03 17:34:27,269648
3,1886662132,https://www.twitch.tv/videos/1886662132,watch_party_krüesports_vs_leviatan_final_del_l...,2h40m14s,2023-07-31 18:58:10,2023-07-31 18:58:10,126700
4,1879835931,https://www.twitch.tv/videos/1879835931,watch_party_krüesports_vs_leviatan_final_del_l...,4h13m28s,2023-07-23 19:47:35,2023-07-23 19:47:35,248242


In [12]:
df.to_csv('twitch_api_slakun10_csv.csv', index=False)